In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv
# from config import api_key

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2017 
# 2015 2013
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
pprint(response)

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'result': {'_links': {'next': '/api/3/action/datastore_search?limit=1000&q=2018&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&offset=1000',
                       'start': '/api/3/action/datastore_search?limit=1000&q=2018&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991'},
            'fields': [{'id': '_id', 'type': 'int'},
                       {'id': 'OBJECTID', 'type': 'text'},
                       {'id': 'Year', 'type': 'text'},
                       {'id': 'State', 'type': 'text'},
                       {'id': 'Agency', 'type': 'text'},
                       {'id': 'Unit ID', 'type': 'text'},
                       {'id': 'Fire Name', 'type': 'text'},
                       {'id': 'Local Incident Number', 'type': 'text'},
                       {'id': 'Alarm Date', 'type': 'text'},
                       {'id': 'Containment Date', 'type': 'text'},
                       {'id': 'Cause', 'type': 

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2023 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2023.columns if col not in columns_to_keep]
df_2023 = df_2023.drop(columns=columns_to_drop)
df_2023 = df_2023.dropna(subset=['Containment Date'])
df_2023.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2018,2/20/2018 12:00:00 AM,2/20/2018 12:00:00 AM,14,0.02049659
1,2018,11/16/2018 12:00:00 AM,11/16/2018 12:00:00 AM,14,9.018476
2,2018,6/24/2018 12:00:00 AM,6/27/2018 12:00:00 AM,7,386.1122
3,2018,8/20/2018 12:00:00 AM,8/20/2018 12:00:00 AM,9,7.95544
4,2018,6/12/2018 12:00:00 AM,6/12/2018 12:00:00 AM,5,63.63537


In [4]:
alarm_date_23 = []
containment_date_23 = []
acres_23 = []
for index, row in df_2023.iterrows():
    alarm_date_23.append(row['Alarm Date'])
    containment_date_23.append(row['Containment Date'])
    acres_23.append(row['GIS Calculated Acres'])
alarm_date_23 = [dt.replace(' 12:00:00 AM', '') for dt in alarm_date_23]
containment_date_23 = [str(item) for item in containment_date_23]
containment_date_23 = [dt.replace(' 12:00:00 AM', '') for dt in containment_date_23]

In [5]:
df_2023['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2023['Alarm Date']]
df_2023['Containment Date'] = [str(item) for item in df_2023['Containment Date']]
df_2023['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2023['Containment Date']]
df_2023.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2018,2/20/2018,2/20/2018,14,0.02049659
1,2018,11/16/2018,11/16/2018,14,9.018476
2,2018,6/24/2018,6/27/2018,7,386.1122
3,2018,8/20/2018,8/20/2018,9,7.95544
4,2018,6/12/2018,6/12/2018,5,63.63537


In [6]:
df_2023 = df_2023.rename(columns={'Cause': 'Cause #'})
df_2023['Cause'] = ''
df_2023['Cause #'] = df_2023['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2023['Cause'] = df_2023['Cause #'].apply(map_cause_to_meaning)
df_2023.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2018,2/20/2018,2/20/2018,14,0.02049659,Unknown / Unidentified
1,2018,11/16/2018,11/16/2018,14,9.018476,Unknown / Unidentified
2,2018,6/24/2018,6/27/2018,7,386.1122,Arson
3,2018,8/20/2018,8/20/2018,9,7.95544,Miscellaneous
4,2018,6/12/2018,6/12/2018,5,63.63537,Debris


In [7]:
df_2023.to_csv("outputs/fires_2017.csv",
                  encoding="utf-8", index=False, header=True)

In [8]:
df_2023.dtypes

Year                    object
Alarm Date              object
Containment Date        object
Cause #                  int32
GIS Calculated Acres    object
Cause                   object
dtype: object

In [9]:
df_2023['Alarm Date'] = pd.to_datetime(df_2023['Alarm Date'])
df_byalarm = df_2023.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,180,2018,2017-12-21,12/21/2017,14,0.01809152,Unknown / Unidentified
1,178,2018,2017-12-22,12/22/2017,14,0.3788415,Unknown / Unidentified
2,158,2018,2018-01-03,1/3/2018,14,0.03201193,Unknown / Unidentified
3,146,2018,2018-01-20,1/20/2018,14,0.6356196,Unknown / Unidentified
4,213,2018,2018-01-23,1/23/2018,14,0.1679536,Unknown / Unidentified


In [10]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2017-12-21 00:00:00')

In [11]:
df_2023['Containment Date'] = pd.to_datetime(df_2023['Containment Date'])
df_bycontainment = df_2023.sort_values(by='Containment Date', ascending = False).reset_index()
latest_containment = df_bycontainment['Containment Date'][0]
df_bycontainment.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,74,2018,2018-08-19,2019-08-19,14,154.5624,Unknown / Unidentified
1,274,2018,2018-12-25,2018-12-25,11,2.18085,Powerline
2,362,2018,2018-10-19,2018-12-18,1,38117.85,Lightning
3,298,2018,2018-07-29,2018-12-18,14,327.5861,Unknown / Unidentified
4,368,2018,2018-07-22,2018-12-18,14,4.837892,Unknown / Unidentified


In [12]:
latest_containment

Timestamp('2019-08-19 00:00:00')

In [13]:
output_path = os.path.join("outputs", "timeline_dates_2017.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])